<a href="https://colab.research.google.com/github/Mono-Blaine/CMPS-261-Project/blob/master/261_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import auc, accuracy_score, mean_squared_error, confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the dataset
# header=None prevents the first row from becoming column names
data = pd.read_csv('drive/MyDrive/HIGGS_train.csv', header=None)

<ipython-input-2-c4aa6816fafd>:3: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('drive/MyDrive/HIGGS_train.csv', header=None)


# **Preprocessing**

**Dirty Data:**
* 490960V: error
* 490961V: s
* 261027I: "1.01"

In [26]:
# removes rows that aren't of type float. however, removes a lot so something wrong may be happening
# for i in range(29):
#     data = data[[not isinstance(value, str) for value in data[i]]]
#     data = data[[isinstance(value, float) for value in data[i]]]

# removes columns that aren't all float
# data2=data2.select_dtypes(include=['float64'])
# data2=data2.select_dtypes(exclude=['object'])

print(data.shape)

def filter(value):
    if isinstance(value, str) and not value.isnumeric():
        try:
            return float(value.replace('"','').replace('\'',''))
        except:
            return np.nan;
    return value

data = data.applymap(filter)

print(data.shape)

# get features
X = data.iloc[:, 1:]
# get output
y = data.iloc[:, 0]

(600000, 29)
(600000, 29)


# **Logistic Regression**

In [27]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

# create the model
model = LogisticRegression(max_iter=10000,  penalty='l2', solver='lbfgs')

# training
model.fit(X_train, y_train)

# predict the result of testing data
y_pred = model.predict(X_test)

# check model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(cross_val_score(model, X, y, cv=5, error_score='raise'))

Accuracy: 0.6420166666666667


ValueError: ignored

# **XGBoost**

In [28]:
# create the XGBoost model
model = XGBClassifier(objective="binary:logistic", random_state=42, max_depth=6, learning_rate=0.1)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# train the model
model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

# predict the result of testing data
y_pred = model.predict(X_test)

# check model performance
print("Accuracy Score:", accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred) 
print("Confusion Matrix")
print(cm)

# do K-fold (5-fold) cross-validation
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(model, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

[0]	validation_0-logloss:0.67555
[1]	validation_0-logloss:0.66107
[2]	validation_0-logloss:0.64889
[3]	validation_0-logloss:0.63831
[4]	validation_0-logloss:0.62918
[5]	validation_0-logloss:0.62126
[6]	validation_0-logloss:0.61454
[7]	validation_0-logloss:0.60861
[8]	validation_0-logloss:0.60294
[9]	validation_0-logloss:0.59853
[10]	validation_0-logloss:0.59395
[11]	validation_0-logloss:0.58978
[12]	validation_0-logloss:0.58614
[13]	validation_0-logloss:0.58290
[14]	validation_0-logloss:0.57971
[15]	validation_0-logloss:0.57684
[16]	validation_0-logloss:0.57411
[17]	validation_0-logloss:0.57197
[18]	validation_0-logloss:0.56955
[19]	validation_0-logloss:0.56740
[20]	validation_0-logloss:0.56557
[21]	validation_0-logloss:0.56366
[22]	validation_0-logloss:0.56185
[23]	validation_0-logloss:0.56044
[24]	validation_0-logloss:0.55877
[25]	validation_0-logloss:0.55736
[26]	validation_0-logloss:0.55603
[27]	validation_0-logloss:0.55489
[28]	validation_0-logloss:0.55372
[29]	validation_0-loglos